In [ ]:
def getNewA(self, n):
    m = n*2 + 1
    sob = self.mSob
    
    L = sob.L
    u_xy = sob.u_xyPlane
    u_xz = sob.u_xzPlane
    
    # Get global xi
    if m > 2:
        dis = 1/(m-1)
        xis_global = [(i+1)*dis for i in range(m-2)]
    xis_global = [0] + xis_global + [1]
    
    # Get local xi
    # and indexes for separations
    xis_local = (np.array(xis_global) % (1/n)) / (1/n)
    xis_local[-1] = 1
    separateIndex = [0]
    for i in range(len(xis_local)):
        if i == 0:
            pass
        else:
            if xis_local[i] <= xis_local[i-1]:
                separateIndex.append(i)
    separateIndex.append(-1)
    
    # Get u in y and z
    uYs = []
    uZs = []
    for xi_value in range(len(xis_global)):
        uY = u_xy.evalf(subs={'xi': xi_value})
        uZ = u_xz.evalf(subs={'xi': xi_value})
        uYs.append(uY)
        uZs.append(uZ)
        
    # Get N
    N = np.zeros((m, 2*(n+1)))
    pts = []
    segL = L/n
    for i in range(len(separateIndex)-1):        
        xis = xis_local[separateIndex[i]:separateIndex[i+1]]
        if separateIndex[i+1] == -1:
            xis = np.append(xis, xis_local[-1])

        if i == n-1:
            N[separateIndex[i]:, i*2:] = SkeletonOfBeam.H(xis, segL)
        else:
            N[separateIndex[i]:separateIndex[i+1], i*2:i*2+4] = SkeletonOfBeam.H(xis, segL)
    
    self.aYs = np.dot(np.linalg.pinv(N), uYs)
    self.aZs = np.dot(np.linalg.pinv(N), uZs)